In [ ]:
import os
import numpy as np

In [ ]:
import energy_model
import utils

In [ ]:
import importlib
importlib.reload(energy_model)
importlib.reload(utils)

In [ ]:
from energy_model import EnergyModel
from utils.data_handling import ScenarioData
from utils.gurobi_env import get_Gurobi_WLS_env

In [ ]:
dataset_dir = os.path.join('data','processed')

In [ ]:
# create test scenario data

ts_dict = {
    'load_level': 3e5,
    'wind_year': 2010,
    'solar_year': 2010,
    'price_year': 2023,
    'carbon_year': 2023
}

cost_dict = { # sense check costs by LCOE
    'wind_capex': 5e3, # €/kWp
    'wind_opex': 100, # €/kWp/yr
    'wind_lifetime': 20, # years
    'solar_capex': 1e3, # €/kWp
    'solar_opex': 10, # €/kWp/yr
    'solar_lifetime': 20, # years
    'carbon_price': 1.0 # €/kgCO2
}

storage_dict = {
    'li-ion': {
        'cost': 300,
        'lifetime': 12,
        'efficiency': 0.9,
        'discharge_ratio': 0.4
    },
    'other': {
        'cost': 100,
        'lifetime': 20,
        'efficiency': 0.5,
        'discharge_ratio': 0.2
    },
    'other2': {
        'cost': 175,
        'lifetime': 20,
        'efficiency': 0.8,
        'discharge_ratio': 0.2
    }
}

In [ ]:
scenario = ScenarioData(ts_dict, cost_dict, storage_dict)
scenario.load_timeseries(dataset_dir)

In [ ]:
settings = {
    'T':8760,
    'initial_SoC':0.5,
    'max_storage_cap':1e9, # slack bound
    'N_technologies': 2,
    'grid_capacity': 500e3, # kW
    'capex_budget': 10e9/20, # €/yr
    'solar_capacity_limit': 500e3, # kWp
}

Note:
1. You need to be careful that the capex budget is sufficiently large to allow the purchase of enough generation capacity to satisfy the plant demand
2. The solar capacity limit is needed to make sure that the optimised solar capacity is not infeasibly large due to land area

In [ ]:
model = EnergyModel()
model.generate_SP([scenario],settings)

In [ ]:
env = get_Gurobi_WLS_env(silence=False)
model.model.solve(env=env)

In [ ]:
print(scenario.load@scenario.elec_prices)

In [ ]:
print(model.model.objective)

In [ ]:
print('wind capacity', model.model.variables.wind_capacity.solution.values)
print('solar capacity', model.model.variables.solar_capacity.solution.values)
print('storage capacities', [getattr(model.model.variables,f'{tech}_capacity').solution.values for tech in model.techs])

In [ ]:
for key,val in model.scen_obj_contrs[0].items():
    print(key,val.solution.values)

In [ ]:
print(sum([model.scen_obj_contrs[0][key] for key in ['wind','solar','storage']]).solution.values)